In [186]:
#local stiffness matrix
import numpy as np
import math as m

E = 50
A = 2
L = 20
theta = 1

#initialize k_local
def define_ke_local(E, A, L):
    k = np.zeros((4,4))
    
    k[0][0] = (E*A/L)
    k[2][2] = (E*A/L)
    k[0][2] = (-E*A/L)
    k[2][0] = (-E*A/L)
    
    return k

def define_rotation(theta):
    unit_rot = [[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]
    beta = np.zeros((4,4))
    beta[0:2,0:2] = unit_rot
    beta[2:4,2:4] = unit_rot
    return beta

def calc_length(i_coords, j_coords):
    return np.linalg.norm((j_coords - i_coords))

def calc_orientation(i_coords, j_coords):
    delta = j_coords - i_coords
    return np.arctan2(delta[1], delta[0])

def calc_dof(i_node_num, j_node_num):
    dof_i = [i_node_num * 2 - 1, i_node_num * 2]
    dof_j = [j_node_num * 2 - 1, j_node_num * 2]
    dof = dof_i + dof_j
    return dof

def define_ke_global(ke_local, beta):
    ke_global = np.dot((np.dot(np.transpose(beta), ke_local)), beta)
    return ke_global


## DISCLAIMER: I had difficulties setting up a loop using python for this problem.
## I was able to set it up using the specific truss case, however.
def assemble_big_mac(num_dof, ke_global, global_dof):
    big_mac = np.zeros((num_dof, num_dof))
    for i in range(len(ke_global)):
        if i == 0 or i == 2:
            big_mac[i:i+4,i:i+4] += ke_global[i]
        elif i == 1:
            big_mac[0:2,0:2] += ke_global[i][0:2,0:2]
            big_mac[0:2,4:6] += ke_global[i][0:2,2:4]
            big_mac[4:6,0:2] += ke_global[i][2:4,0:2]
            big_mac[4:6,4:6] += ke_global[i][2:4,2:4]
    return big_mac



In [188]:
nodes = np.array([[0.0, 0.0], [-10.0,-10.0], [10.0, -10.0]])
mat_props = [29000000.0, 10000.0, 15000000.0]
section_props = [1.0, 10.0, 3.6]
elements = [[0, 1, 0, 1], [0, 2, 2, 0], [1, 2, 1, 2]]
#print(len(elements))
A = [section_props[elements[i][3]] for i in range(len(elements))]
#print(A)
E = [mat_props[elements[i][2]] for i in range(len(elements))]
#print(E)
L = [calc_length(nodes[elements[i][0]], nodes[elements[i][1]]) for i in range(len(elements))]
#print(L)
theta = [calc_orientation(nodes[elements[i][0]], nodes[elements[i][1]]) for i in range(len(elements))]
#print(theta)
ke_local = [define_ke_local(E[i], A[i], L[i]) for i in range(len(L))]
#print(ke_local)
beta = [define_rotation(theta[i]) for i in range(len(theta))]
#print(beta)
ke_global = [define_ke_global(ke_local[i], beta[i]) for i in range(len(ke_local))]
#print(ke_global)
element_global_dof = [calc_dof(elements[i][0], elements[i][1]) for i in range(len(elements))]
#print(element_global_dof)
num_dof = np.shape(nodes)[0] * 2
K = assemble_big_mac(num_dof, ke_global, element_global_dof)


print("Local Stiffnesses:\n")
print("Element 1:", ke_local[0], "\n")
print("Element 2:", ke_local[1], "\n")
print("Element 3:", ke_local[2], "\n")
print("Global Stiffnesses:\n")
print("Element 1:", ke_global[0], "\n")
print("Element 2:", ke_global[1], "\n")
print("Element 3:", ke_global[2], "\n")
print("Big Mac: \n")
print(K)

[[-1, 0, 1, 2], [-1, 0, 3, 4], [1, 2, 3, 4]]
Local Stiffnesses:

Element 1: [[ 20506096.65440988         0.         -20506096.65440988
          0.        ]
 [        0.                 0.                 0.
          0.        ]
 [-20506096.65440988         0.          20506096.65440988
          0.        ]
 [        0.                 0.                 0.
          0.        ]] 

Element 2: [[ 1060660.17177982        0.         -1060660.17177982        0.        ]
 [       0.                0.                0.                0.        ]
 [-1060660.17177982        0.          1060660.17177982        0.        ]
 [       0.                0.                0.                0.        ]] 

Element 3: [[ 1800.     0. -1800.     0.]
 [    0.     0.     0.     0.]
 [-1800.     0.  1800.     0.]
 [    0.     0.     0.     0.]] 

Global Stiffnesses:

Element 1: [[ 10253048.32720494  10253048.32720494 -10253048.32720494
  -10253048.32720494]
 [ 10253048.32720494  10253048.32720494 -1025304